In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn import metrics
import env
from wrangle import *
import socket

In [2]:
df = wrangle_logs()

In [3]:
df.head()

,time,path,user_id,cohort_id,ip
date,,,,,
2018-01-26,2023-02-21 09:55:03,/,1,8.0,97.105.19.61
2018-01-26,2023-02-21 09:56:02,java-ii,1,8.0,97.105.19.61
2018-01-26,2023-02-21 09:56:05,java-ii/object-oriented-programming,1,8.0,97.105.19.61
2018-01-26,2023-02-21 09:56:06,slides/object_oriented_programming,1,8.0,97.105.19.61
2018-01-26,2023-02-21 09:56:24,javascript-i/conditionals,2,22.0,97.105.19.61


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 900223 entries, 2018-01-26 to 2021-04-21
Data columns (total 5 columns):
 #   Column     Non-Null Count   Dtype         
---  ------     --------------   -----         
 0   time       900223 non-null  datetime64[ns]
 1   path       900222 non-null  object        
 2   user_id    900223 non-null  int64         
 3   cohort_id  847330 non-null  float64       
 4   ip         900223 non-null  object        
dtypes: datetime64[ns](1), float64(1), int64(1), object(2)
memory usage: 41.2+ MB


In [5]:
df.ip.value_counts()

97.105.19.58       284579
97.105.19.61        61662
192.171.117.210      9515
71.150.217.33        6791
76.185.145.231       4754
                    ...  
172.58.109.140          1
99.203.213.59           1
99.203.212.173          1
173.239.198.246         1
72.181.124.244          1
Name: ip, Length: 5531, dtype: int64

In [7]:
def dns_ptr_lookup(addr):
    try:
        return socket.gethostbyaddr(addr)
    except socket.herror:
        return None, None, None

In [8]:
ip_list = []
for item in np.unique(df.ip):
    output = dns_ptr_lookup(str(item))
    ip_item = {'ip':item,
               'name':output[0]}
    ip_list.append(ip_item)

In [9]:
ip_df = pd.DataFrame(ip_list)
df = df.merge(ip_df, left_on = 'ip', right_on = 'ip')

In [12]:
df.name.value_counts()

097-105-019-058.biz.spectrum.com                 284579
097-105-019-061.biz.spectrum.com                  61662
71-150-217-33.lightspeed.rcsntx.sbcglobal.net      6791
cpe-76-185-145-231.satx.res.rr.com                 4754
host194.regusus.com                                4360
                                                  ...  
mobile-107-77-217-18.mobile.att.net                   1
ip-99-203-155-142.pools.spcsdns.net                   1
65-43-115-99.lightspeed.snantx.sbcglobal.net          1
cpe-70-117-30-148.satx.res.rr.com                     1
gen-070-125-229-227.biz.spectrum.com                  1
Name: name, Length: 3815, dtype: int64

In [10]:
hourly_hits = df['ip'].resample('60min', on=df['time']).count()

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().